In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'preprocessed_data.csv')

In [3]:
df = pd.read_csv(data_path)
df.head()

,bedroom,bathroom,beds,guests,wifi,tv,cable_tv,ac,workspace,hot_water,...,lng,distance_to_coastline,room_name,booking_window,stay_duration_in_days,review_sentiment_score,rating,booking_earned,average_daily_rate,rating_rounded
0,1,1,1,2,1,0,0,1,1,1,...,115.113378,15.6035,Bingin Ombak Apartment - 1 Lantai 1,81,4,0.458603,4.0,5911197.97,1.477799e+06,4.0
1,1,1,1,2,1,0,0,1,1,0,...,115.113378,15.6035,Bingin Ombak Apartment - 2 Lantai 2,81,4,0.458603,4.0,5911197.97,1.477799e+06,4.0
2,1,2,1,2,1,0,0,0,1,0,...,115.113378,15.6035,Standard (PLEASE IGNORE),81,4,0.455450,4.0,5911197.97,1.477799e+06,4.0
3,1,2,1,2,1,0,0,0,1,0,...,115.113378,15.6035,Standard (PLEASE IGNORE),81,4,0.455450,4.0,5911197.97,1.477799e+06,4.0
4,1,3,1,2,1,0,0,0,1,0,...,115.113378,15.6035,Standard (PLEASE IGNORE),81,4,0.452298,4.0,5911197.97,1.477799e+06,4.0


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

numeric_columns = df.select_dtypes(include=['number'])
numeric_df = numeric_columns.drop(columns=['average_daily_rate'])

# Separate features and target variable
X = numeric_df.drop(columns=['booking_earned'])  # Features
y = numeric_df['booking_earned']  # Target variable

# Melakukan standarisasi pada fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

# Inisialisasi model Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor(random_state=42)

# Definisi grid parameter yang akan dijelajahi
param_grid = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}

# Membuat objek GridSearchCV
grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5, verbose=2)

# Melatih model pada data
grid_search.fit(X_train, y_train)

# Mendapatkan parameter terbaik
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Mendapatkan model terbaik
best_gb_regressor = grid_search.best_estimator_

# Memprediksi pada set pengujian
y_pred = best_gb_regressor.predict(X_test)

# Menghitung MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   7.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   8.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   7.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   8.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   8.7s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=  39.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=  38.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=  36.6s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=  44.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=  45.5s
[CV] END .learning_rate=0.01, max_depth=3, n_estimators=1000; total time= 1.8min
[CV] END .learning_rate=0.01, max_depth=3, n_es